In [1]:
import numpy as np
import cvxpy as cp

In [2]:
# parameter estimates.
alpha = 0.00016       # watts / (units P * percent U1)
P1 = 200              # P units
P2 = 100              # P units
CpH = 4.46            # heat capacity of the heater (J/deg C)
CpS = 0.819           # heat capacity of the sensor (J/deg C)
Ua = 0.050            # heat transfer coefficient from heater to environment
Ub = 0.021            # heat transfer coefficient from heater to sensor
Uc = 0.0335           # heat transfer coefficient between heaters
Tamb = 21             # ambient room temperature

# state space model
A = np.array([[-(Ua + Ub + Uc)/CpH, Ub/CpH, Uc/CpH, 0], 
              [Ub/CpS, -Ub/CpS, 0, 0],
              [Uc/CpH, 0, -(Ua + Ub + Uc)/CpH, Ub/CpH],
              [0, 0, Ub/CpS, -Ub/CpS]])

Bu = np.array([[alpha*P1/CpH, 0], [0, 0], [0, alpha*P2/CpH], [0, 0]])

Bd = np.array([[Ua/CpH], [0], [Ua/CpH], [0]])

C = np.array([[0, 1, 0, 0], [0, 0, 0, 1]])

# initial values for states and inputs
u_initial = np.array([0, 0])
d_initial = np.array([Tamb])
x_initial = np.array([Tamb, Tamb, Tamb, Tamb])

In [3]:
# knowns
d = d_initial            # disturbance
r = np.array([30, 40])   # setpoints

# unknowns to be computed
u = cp.Variable(2)
x = cp.Variable(4)
y = cp.Variable(2)

# objective
objective = cp.Minimize(0)

# model constraints
model_constraints = [
    0 == A@x + Bu@u + Bd@d,
    y == C@x]

# input constraints

input_constraints = [
    0 <= u, 
    u <= 100]

# output constraints
output_constraints = []

# setpoints
r = np.array([70, 40])
setpoints = []
objective = cp.Minimize(0)

# solve problem
constraints = model_constraints + input_constraints + output_constraints + setpoints
problem = cp.Problem(objective, constraints)
problem.solve()

# display solution
print(f"u = {u.value}")
print(f"x = {x.value}")
print(f"y = {y.value}")

u = [49.99999592 49.9999971 ]
x = [48.41880129 48.41880129 41.58119517 41.58119517]
y = [48.41880129 41.58119517]
